# Tornado Heuristics

In [1]:
import pandas as pd

tornado_transactions = pd.read_csv('../data/tornado_trx_transfers.csv', index_col=[0])
intra_set_asset_transfers = pd.read_csv('../data/all_intra_transfers.csv', index_col=[0], low_memory=False)

In [2]:
tornado_eth = tornado_transactions[tornado_transactions['tokenName']=='Ether'].copy()

# Find deposit addresses for ETH pools
deposit_eth = tornado_eth[tornado_eth['functionName']=='deposit(bytes32 _id)']['from'].tolist()

# Find withdrawal addresses for ETH pools
withdraw_eth = tornado_eth[tornado_eth['functionName']=='withdraw(bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund)']['from'].tolist()

tornado_eth['transaction_type'] = tornado_eth['functionName'].apply(lambda x: 'deposit' if x == 'deposit(bytes32 _id)' else 'withdraw' if x == 'withdraw(bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund)' else None)

In [3]:
tornado_token = tornado_transactions[tornado_transactions['tokenName']!='Ether'].copy()

# Find deposit addresses for other token pools
tornado_token['transaction_type'] = tornado_token['isSet'].apply(lambda x: 'withdraw' if x == 'to' else 'deposit' if x == 'from' else None)

# Find withdrawal addresses for other token pools
withdraw_token = tornado_token[tornado_token['isSet']=='to']['to'].tolist()
deposit_token = tornado_token[tornado_token['isSet']=='from']['from'].tolist()

In [4]:
tornado_transactions

,hash,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,tokenName,tokenType,isSet,userAddress,contractAddress,pool
0,0xab9112a0bfd7ee489f32ad2eace95cd9111da9091008...,0x007f44362400de9f364efa919ff84c6ea9e210d5,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1597667681,171.0,1.000000e+17,112000000000,0xb214faa52606de7c18da92872e783b4cf27c757334ca...,966887,deposit(bytes32 _id),Ethereum,Ether,native,from,0x007f44362400de9f364efa919ff84c6ea9e210d5,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
1,0xb96895d541a818232300808230ff202f8e1217609f10...,0x1dd5df3b2d78271039f3dc39eb9350ecc31ca9dc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1605348173,39.0,1.000000e+17,23000000000,0xb214faa50bf6fe5e24e3223b7e2cec17a028597694c1...,978703,deposit(bytes32 _id),Ethereum,Ether,native,from,0x1dd5df3b2d78271039f3dc39eb9350ecc31ca9dc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
2,0x282777a6664cf1d9d3f1695bed549f7a62ec57411350...,0x1ec594a869dc67d78fabc33963d3c6a0ab017dd6,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1602676051,827.0,1.000000e+17,33000000000,0xb214faa512dd5f64b38ff2414a7e7cf5219e5ff98ec6...,978703,deposit(bytes32 _id),Ethereum,Ether,native,from,0x1ec594a869dc67d78fabc33963d3c6a0ab017dd6,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
3,0xc880becfb7f3a939765f1cb650cd7b624cdb0a046a22...,0x2fef65e4d69a38bf0dd074079f367cdf176ec0de,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1603804997,7601.0,1.000000e+17,45000000000,0xb214faa50205af153fb0c6e1818a2e396450ed9baf14...,972795,deposit(bytes32 _id),Ethereum,Ether,native,from,0x2fef65e4d69a38bf0dd074079f367cdf176ec0de,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
4,0xd9eeeec262897f88c4323013ca927cb3cb94e7f22347...,0x30602250c5f1fcba5407e99b1dfaab992ea4ffd2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1583900510,4830.0,1.000000e+17,13000000000,0xb214faa5068a8346b266fcdfd7a9830a2409eac71681...,984611,deposit(bytes32 _id),Ethereum,Ether,native,from,0x30602250c5f1fcba5407e99b1dfaab992ea4ffd2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0x5607c93ba2fc0814327af6f53b53923bcc747edc7b2b...,0x169ad27a470d064dede56a2d3ff727986b15d52b,0x3f0500b79c099dfe2638d0faf1c03f56b90d12d1,1602750994,2787.0,7.718709e+01,80000000000,NaN,361300,NaN,Ethereum,Tether USD,20,to,0x3f0500b79c099dfe2638d0faf1c03f56b90d12d1,0x169ad27a470d064dede56a2d3ff727986b15d52b,100 USDT
80,0xdb5ea6e7a175dda3b9359910aaa5ebf3f70b5e2ab253...,0x178169b423a011fff22b9e3f3abea13414ddd0f1,0x0c40bff248ed5d99d294ceca8826c7924f745134,1619556022,3439.0,9.831973e-02,58650000000,NaN,399260,NaN,Ethereum,Wrapped BTC,20,to,0x0c40bff248ed5d99d294ceca8826c7924f745134,0x178169b423a011fff22b9e3f3abea13414ddd0f1,0.1 wBTC
81,0x7bce45c79f340555a9c7e66116875cb56ad065a24488...,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0x7a218b62ae9e23aac788979ead8be51258a3cbc5,1621903058,154.0,1.000000e+03,46000000000,NaN,395784,NaN,Ethereum,Dai Stablecoin,20,to,0x7a218b62ae9e23aac788979ead8be51258a3cbc5,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,1k Dai
82,0x9b7a76858cc9ca49de2cb273e4a2e635e4b652019acf...,0x62051bfd3a4f7039a849142e6e5ea172cbda5949,0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3,1598285598,212.0,1.000000e+02,128000000000,NaN,1003512,NaN,Ethereum,Dai Stablecoin,20,from,0x62051bfd3a4f7039a849142e6e5ea172cbda5949,0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3,100 Dai


### Heuristic 1 - Address Match
No user deposited to and withdrew from a Tornado pool with the same address. The code below is simplified, as it does not check by pool.

In [5]:
address_match_eth = list(set(withdraw_eth) & set(deposit_eth))
print(address_match_eth)

address_match_token = list(set(withdraw_token) & set(deposit_token))
print(address_match_token)

[]
[]


### Heuristic 2 - Unique Gas Price

There are no very specific gas prices with a count above 1.

In [6]:
tornado_concat = pd.concat([tornado_eth, tornado_token], ignore_index=True)

gas_prices = tornado_concat.groupby(['gasPrice']).size().reset_index(name='count')[['gasPrice', 'count']]

filtered_gas_prices = gas_prices.query('count >= 2')
filtered_gas_prices

,gasPrice,count
0,1000000000,3
1,1010000000,2
11,11000000000,7
12,11200000000,2
14,13000000000,7
15,15100000000,2
18,21000000000,3
25,25000000000,2
26,26000000000,3
28,33000000000,2


### Heuristic 3 - Linked Addresses
No interaction between deposit and withdrawal addresses for both the withdrawal and the deposit addresses.

In [7]:
# take intra-set asset transfer dataframe
# check if there are transfers where both the from and to are in the subsets
filtered_df = intra_set_asset_transfers[
    ((intra_set_asset_transfers['from'].isin(withdraw_eth)) & (intra_set_asset_transfers['to'].isin(deposit_eth))) |
    ((intra_set_asset_transfers['from'].isin(deposit_eth)) & (intra_set_asset_transfers['to'].isin(withdraw_eth)))
]
filtered_df

,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,contractAddress,tokenName,tokenType,tokenID
hash,,,,,,,,,,,,,,


In [8]:
filtered_df = intra_set_asset_transfers[
    ((intra_set_asset_transfers['from'].isin(withdraw_token)) & (intra_set_asset_transfers['to'].isin(deposit_token))) |
    ((intra_set_asset_transfers['from'].isin(deposit_token)) & (intra_set_asset_transfers['to'].isin(withdraw_token)))
]
filtered_df

,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,contractAddress,tokenName,tokenType,tokenID
hash,,,,,,,,,,,,,,


### Heuristic 4 - Multiple Denomination
No address withdrew from a specific pool more than once, therefore, this heuristic is not applicable.